In [1]:
!pip install nltk
!pip install --upgrade numpy
!pip install multiprocesspandas
import pandas as pd
import html
import re
import time
import numpy as np
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz  
import nltk
nltk.download('punkt')
from multiprocesspandas import applyparallel
from nltk.corpus import stopwords
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
train_df = pd.read_csv('C:\\Projects\\natural_lang_proc\\train_tweets.csv')
test_df = pd.read_csv('C:\\Projects\\natural_lang_proc\\test_tweets.csv')
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df = combine_df.fillna(0)
combine_df['clean_tweet'] = combine_df['tweet'].apply(lambda x: html.unescape(x))

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty


In [8]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49159 non-null  int64  
 1   label        49159 non-null  float64
 2   tweet        49159 non-null  object 
 3   clean_tweet  49159 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.5+ MB
None


In [9]:
combine_df['tweet'] = combine_df['tweet'].astype('string')
combine_df['clean_tweet'] = combine_df['clean_tweet'].astype('string')

In [10]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49159 non-null  int64  
 1   label        49159 non-null  float64
 2   tweet        49159 non-null  string 
 3   clean_tweet  49159 non-null  string 
dtypes: float64(1), int64(1), string(2)
memory usage: 1.5 MB
None


In [13]:
def delete_user():
    pattern = r"@[\w]*"
    combine_df['clean_tweet'] = [re.sub(pattern, ' ', fname) for fname in combine_df['tweet']]
    #return combine_df['clean_tweet']
np.vectorize(delete_user())

In [15]:
combine_df['clean_tweet'] = combine_df['clean_tweet'].str.lower()

In [16]:
obj = combine_df['clean_tweet'].copy()
def replace_aint_cant():
    apostrophe_dict = dict([
    ("ain't", "am not"),
    ("can't", "can not"),
    (":)", "happy")])
    #obj = combine_df['clean_tweet'].copy()
    for k,v in apostrophe_dict.items(): #based on the rules in the dict
        rule1 = (r"(\b)(%s)(\b)" % k)
        rule2 = (lambda m: apostrophe_dict.get(m.group(), m.group())) #found this online, no idea wtf this does but it works
        obj = obj.str.replace(rule1, rule2, regex=True, flags=re.IGNORECASE) #use flags here to avoid the dictionary iteration problem
combine_df['clean_tweet'] = obj

In [18]:
pattern = r'[^\w\s]'
combine_df['clean_tweet'] = [re.sub(pattern, ' ', fname) for fname in combine_df['tweet']]

In [19]:
pattern = r'[^a-zA-Z0-9]'
combine_df['clean_tweet'] = [re.sub(pattern, ' ', fname) for fname in combine_df['tweet']]

In [20]:
pattern = r'[^a-zA-Z]'
combine_df['clean_tweet'] = [re.sub(pattern, ' ', fname) for fname in combine_df['tweet']]

In [21]:
obj = combine_df['clean_tweet'].copy()
for i in obj:
    z = ' '.join([w for w in i.split() if len(w)>1])
    combine_df['clean_tweet'] = combine_df['clean_tweet'].replace(i, z)

In [25]:
combine_df['tweet_token'] = combine_df['clean_tweet'].apply_parallel(word_tokenize) 

In [31]:
from multiprocesspandas import applyparallel
stop_words = set(stopwords.words("english"))

In [32]:
stop_words = set(stopwords.words("english"))
combine_df['tweet_token_filtered'] = combine_df['tweet_token'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))   

In [41]:
wordnet_lemmatizer = WordNetLemmatizer()

def convert(text):
    lemmatized_text = []
    for i in text:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(i,pos="v")))

    return ' '.join(lemmatized_text)

combine_df['tweet_lemmatized'] = combine_df['tweet_token'].apply(lambda x: convert(x))

In [44]:
porter_stemmer = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

combine_df['tweet_stemmed'] = combine_df['tweet_token'].apply(stem_sentences)

In [46]:
combine_df.to_pickle('combine_df.pkl')